In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
import tiktoken
import emoji

## add module path
sys.path.insert(1, os.path.join( os.getcwd(), ".." ) )

## tokens counter

In [2]:
from src.utils.openaiwrapper.openai_wrapper import token_counter

2025-04-06 21:01:34.789 | INFO     | src.utils.openaiwrapper.openai_wrapper:<module>:17 - Update cost from langchain
2025-04-06 21:01:35.591 | INFO     | src.utils.openaiwrapper.get_openai_cost:extract_and_save_model_cost_data:106 - Model cost data saved to c:\Users\caanp\OneDrive\MBA\Repositorio\GenAI-MBA\src\utils\openaiwrapper\model_costs.json


In [3]:
enc = tiktoken.get_encoding("cl100k_base")

hi = enc.encode("❤️")
hi

[49633, 97, 31643]

In [4]:
[enc.decode_single_token_bytes(token) for token in hi]

[b'\xe2\x9d', b'\xa4', b'\xef\xb8\x8f']

## Compressão usando LZMA

In [ ]:
import lzma

def compress_data(input_file, output_file):
    with open(input_file, 'rb') as f_in:
        with lzma.open(output_file, 'wb', preset = 9) as f_out:
            f_out.write(f_in.read())

def decompress_data(input_file, output_file):
    with lzma.open(input_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            f_out.write(f_in.read())

# Exemplo de uso
compress_data('./../Dados/reviews_train.parquet', './../Dados/reviews_train.parquet.xz')
# decompress_data('path/to/input_file.xz', 'path/to/output_file')

## Reading ...

1. Duplicity check. There are some lines ducplicated

Fonte de dados: https://www.kaggle.com/competitions/anlise-de-sentimento-do-google-play/data?select=reviews_train.csv

In [ ]:
## Salvando em formato binário
# data = pd.read_csv("./../Dados/reviews_train.csv")
# data.to_parquet("./../Dados/reviews_train.parquet", index=False)

In [5]:
data = pd.read_parquet("./../Dados/reviews_train.parquet")
data.fillna({"content":""}, inplace=True)
data.shape

(1010490, 14)

In [6]:
## Nulos
sum(pd.read_parquet("./../Dados/reviews_train.parquet")["content"].isna())

24

In [7]:
data["dup"] = data.groupby("reviewId")["reviewId"].transform("count")

In [8]:
data[data["dup"]>1].sort_values("reviewId")[["reviewId"]].drop_duplicates()

,reviewId
43708,00003f3e-6a79-405e-9d4d-a38006d13508
676308,00006049-4815-47b5-8fb6-92ecd383978e
718246,0000bef7-16af-4fa8-a337-ca73cee3cfa4
787984,0000e42e-8066-4359-bdb1-5ac7e82a9744
714824,0000f8b5-a576-4b56-a49a-0c11aaf9d283
...,...
592032,ffff7f38-9725-4a76-9671-73d01022f886
703468,ffff9275-f662-440a-9218-ab9845ce300b
774936,ffff993f-0049-4650-bb56-1b5e510c4800
743647,ffffe9ad-a9f6-4410-8966-2f972a102abf


In [9]:
data.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt', 'appVersion', 'sortOrder', 'appId', 'sentiment', 'dup'],
      dtype='object')

In [10]:
pd.to_datetime(data["at"]).min(), pd.to_datetime(data["at"]).max()

(Timestamp('2013-07-24 09:48:19'), Timestamp('2023-09-26 20:43:38'))

## 1. Fields to Exploratory Analysis

The Reviews in google play store are limited to 350 characters. In the answer.

- Fields selection.
- Remove duplicates.

### 1.1 sentiment <br>

- the "sentiment" field is defined by "the score" field (stars).
    - 0 (Negative): 1 ou 2 stars
    - 1 (Neutral): 3 stars
    - 2 (Positive): 4 ou 5 stars
<br>
> however, there may be revies with text expressing negative sentiment tha have 4 or 5 stars<br>
>> - Ex: ReviewId='3d3c0447-f012-4dec-a757-df8f310a9ee7<br><br>
       `Dei 5 entrela para fica em primeiro"Horrivel vendi o produto to tentando receber nao libera meu dinheiro ,demora pra liberar, e ainda eles diz q vou receber o valor por parte, se for asim eu entregava o produto por parte entao affs vou procurar meus diretos ... Pq nao e bem asim que funsona se eu vendi pq eu preciso do produto . se fosse pra mim receber por parte tinha entregado as coisas por parte ... Quero meu dinheiro liberado pq vou procura meus direito`

<br>

- There are some reviews with thumb (👍) or heart face (😍) but with 1,2 and 3 stars which correspond to negative and neutral sentiment.

In [11]:
from src.utils.ea.exp_funcs import  emojis_extractions

#### Pricing

In [ ]:
## remove duplicates
dados = data[["reviewId","at","content","score","reviewCreatedVersion","replyContent","repliedAt","appVersion","appId","sentiment"]].drop_duplicates()
dados["qtd_tokens"] = dados["content"].apply(lambda x: token_counter(x))
dados["qtd_characters"] = dados["content"].apply(lambda x: len(x))
dados["qtd_words"] = dados["content"].apply(lambda x: len(x.split()))
dados["list_emojis"] = dados["content"].apply(lambda x: emojis_extractions(x) )
dados["qtd_emojis"] = dados["list_emojis"].apply(lambda x: len(x))
dados["with_emojis"] = dados["qtd_emojis"].apply(lambda x: True if x>0 else False)

## Pricing
one_million = 1000000
dados["input_$_gpt_35"] = dados["qtd_tokens"]*0.5/one_million
dados["input_$_gpt_4o"] = dados["qtd_tokens"]*2.5/one_million ## modelo na versão mais recente
# dados["input_$_gpt_4o"] = dados["qtd_tokens"]*5/one_million
dados["input_$_gpt_4o-mini"] = dados["qtd_tokens"]*0.15/one_million
dados["input_$_gpt_4_turbo"] = dados["qtd_tokens"]*10/one_million
dados["input_$_gpt_4"] = dados["qtd_tokens"]*30/one_million


## Saving
dados.to_parquet("./../Dados/reviews_train_EA.parquet", index=False)

dados.shape, data.shape

((729349, 21), (1010490, 15))

In [13]:
dados["appId"].value_counts(dropna=False)

appId
com.mercadolibre                       98007
com.shopee.br                          95797
com.schibsted.bomnegocio.androidApp    84463
com.luizalabs.mlapp                    81605
com.b2w.americanas                     80826
com.alibaba.aliexpresshd               78433
com.zzkko                              64780
com.novapontocom.casasbahia            63128
br.com.enjoei.app                      50682
com.amazon.mShop.android.shopping      31628
Name: count, dtype: int64

In [14]:
data.query(f"appId=='com.mercadolibre'")["content"].str.len()

211203    344
211204    359
211205    497
211206    364
211207    497
         ... 
331198    178
331199     40
331200      9
331201     24
331202     16
Name: content, Length: 120000, dtype: int64

In [15]:
data[["score","sentiment",]].value_counts()

score  sentiment
3      1            266724
5      2            200000
1      0            199950
4      2            184670
2      0            159146
Name: count, dtype: int64

In [16]:
data[data["sentiment"]==1].head(2)

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId,sentiment,dup
30626,71d23587-713d-4d13-b397-78540025aaff,Isabella de Almeida Carvalho,https://play-lh.googleusercontent.com/a-/ALV-U...,"O app é realmente muito bom, utilizo ele a mai...",3,14485,8.3.4,2022-07-28 18:38:42,None,None,8.3.4,most_relevant,com.zzkko,1,2
30627,60080999-ff54-4faa-adcc-044d5fd4443a,Vaniele Guimaraes do Val,https://play-lh.googleusercontent.com/a-/ALV-U...,Pagar fretes diferentes para entrega de uma me...,3,13968,8.6.8,2022-11-10 08:15:35,"Olá, obrigada pela sua avaliação. Caso tenha a...",2022-11-10 23:22:51,8.6.8,most_relevant,com.zzkko,1,2


In [17]:
joinha = dados.query("content=='😍'").reset_index(drop=True)

joinha["score"].value_counts(dropna=False)

score
5    19
4    12
3    11
1     3
2     3
Name: count, dtype: int64

In [18]:
joinha = dados.query("content=='👍'").reset_index(drop=True)

joinha["score"].value_counts(dropna=False)

score
4    117
5    103
3     90
2     12
1      3
Name: count, dtype: int64

# 2. Sampling

For sampling the data, an analysis of the review is required. This analysis will be done using PowerBI.

> **Strategy** <br>
> - Analize OpenAI cost by model
> - Analize text using the total tokens, total characters and total words.
> - Select and sample the text using stratify sampling by sentiment score.


<br>

> **EA Results** <br>
> - There are some texts with a large total tokens, for example, 20 tokens, but they do not have words, so they are emojis.
> - A Emoji or sequence of Emoji represent a single word, however these can be represented by more than one token.

<br>

> **Filter** <br>
> - The best approach to sampling the data is to use the number of word in the reviews.
> - Filter by appId. Select the mercado livre reviews, as it has the most reviews.
> - Sampling data by text with more than one word



In [19]:


from src.utils.sample_process.sampling import Sampling


### 2.1 Filter

In [20]:
mercado_livre = dados.query("appId=='com.mercadolibre' and qtd_words>0")

In [21]:
pd.to_datetime(mercado_livre["at"]).min(), pd.to_datetime(mercado_livre["at"]).max()

(Timestamp('2018-09-12 08:13:02'), Timestamp('2023-09-26 20:00:02'))

In [22]:
sample_10 = Sampling(mercado_livre, random_state = 0).stratified_sample(column="score").reset_index(drop=True)
sample_1 = Sampling(mercado_livre, perc=0.01, random_state = 0 ).stratified_sample(column="score").reset_index(drop=True)

In [23]:
mercado_livre.shape[0], sample_10.shape[0], sample_1.shape[0]

(98006, 9799, 978)

In [24]:
mercado_livre["score"].value_counts(normalize=True).reset_index()\
.merge(mercado_livre["score"].value_counts().reset_index(), on= "score")

,score,proportion,count
0,3,0.308655,30250
1,5,0.185815,18211
2,4,0.178050,17450
3,1,0.166786,16346
4,2,0.160694,15749


In [25]:
sample_10["score"].value_counts(normalize=True).reset_index()\
.merge(sample_10["score"].value_counts().reset_index(), on= "score")

,score,proportion,count
0,3,0.308705,3025
1,5,0.185835,1821
2,4,0.178079,1745
3,1,0.166752,1634
4,2,0.160629,1574


In [26]:
sample_1["score"].value_counts(normalize=True).reset_index()\
.merge(sample_1["score"].value_counts().reset_index(), on= "score")

,score,proportion,count
0,3,0.308793,302
1,5,0.186094,182
2,4,0.177914,174
3,1,0.166667,163
4,2,0.160532,157


In [27]:
sample_1["sentiment"].value_counts(normalize=True).reset_index()\
.merge(sample_1["sentiment"].value_counts().reset_index(), on= "sentiment")

,sentiment,proportion,count
0,2,0.364008,356
1,0,0.327198,320
2,1,0.308793,302


### 2.2 pricing

- for 10% the cost is aprox. USD$17 (9799 reviews)
- for 1% the cost is aprox. USD$1.7 (978 reviews)


In [35]:
from src.utils.ea.exp_funcs import pricing

In [36]:
by_model_10, total_10 = pricing(sample_10)
by_model_10

,tokens ($USD),lines
modelo,,
input_$_gpt_35,0.195767,9799
input_$_gpt_4,11.746020,9799
input_$_gpt_4_turbo,3.915340,9799
input_$_gpt_4o,0.978835,9799
input_$_gpt_4o-mini,0.058730,9799


In [37]:
total_10

tokens ($USD)       16.894692
lines            48995.000000
dtype: float64

In [38]:
by_model_1, total_1 = pricing(sample_1)
by_model_1

,tokens ($USD),lines
modelo,,
input_$_gpt_35,0.019223,978
input_$_gpt_4,1.153380,978
input_$_gpt_4_turbo,0.384460,978
input_$_gpt_4o,0.096115,978
input_$_gpt_4o-mini,0.005767,978


In [39]:
total_1

tokens ($USD)       1.658945
lines            4890.000000
dtype: float64

### 2.3 Save

In [40]:
sample_10.to_parquet("./../Dados/sample_10_Meli.parquet", index=False)

In [41]:
sample_1.to_parquet("./../Dados/sample_1_Meli.parquet", index=False)